In [13]:
import sys
import importlib
importlib.reload(sys)
sys.path.insert(0,'../')
import numpy as np
import netket as nk
from scripts import functions as f
from scripts import new_dynamics 
import os
currentpath = os.getcwd()
parentdir = os.path.dirname(currentpath)

In [3]:
from conf import *
from numba import njit, jit

In [38]:
P = np.load('/home/keisuke/Documents/Research/sshfs/DimerMaster/save/dynamics/h=1.0V=0.9l=[12, 12]/P_n=1.0e+04_0.npy')

In [39]:
T = np.load('/home/keisuke/Documents/Research/sshfs/DimerMaster/save/dynamics/h=1.0V=0.9l=[12, 12]T_n=1.0e+04.npy')

In [45]:
T[-1]

array([2.38896301, 0.45164621, 2.57945529, ..., 0.49767671, 0.68352363,
       0.32363048])

In [7]:
import multiprocessing as mp
N = samples_state.shape[0]
queue = []
process = []
n = 12
index = np.round(np.linspace(0, N, n + 1)).astype(np.int)

In [46]:
t_list = np.arange(0, 32, 1)

In [31]:
t_list = np.linspace(0,2,10**3 )

In [48]:
@njit
def cal_mels(P, T, t_list):
    mels_list = np.zeros((t_list.shape[0],)+P.shape[1:] , dtype=P.dtype)
#     num_samples = np.zeros_like(T, dtype=nb.boolean)
#     sections = np.zeros(t_list.shape[0])
    t_list_ = t_list.reshape(-1,1)

    S = 0
    for n in range(T.shape[0]-1):
        
#         A = T <= t_list[n]
#         num_samples = np.logical_or(A, )
#         ind1, ind2 = np.where(np.logical_xor(A[1:,:],A[:-1,:]))
#     #     sections[n] = S
#         n_state = ind1.shape[0]

        index = np.logical_and(T[n] <= t_list_,  t_list_ < T[n+1])

        for i in range(index.shape[1]):
            mels_list[index[:,i],i] = P[n,i]
    return mels_list

In [49]:
P_list= cal_mels(P, T, t_list)

In [56]:
zero_index = (P_list[:,:,0] == 0) 

In [12]:
P_list.shape

(201, 9984, 288)

In [24]:
P_list.reshape(-1,P_list.shape[-1]).shape

(998400, 288)

In [57]:
P_list_ = P_list.reshape(-1,P_list.shape[-1])
sections = np.zeros(P_list_.shape[0])

In [133]:
num_samples = (P_list[:,:,0]!=0).sum(axis=1)

array([10000, 10000, 10000, ...,  7996,  7975,  7952])

In [126]:
mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1])

In [40]:
hex_.edges_from_hex_(l = hex_.all_hex_index, color=True, num =True)[0].shape

(16, 6, 2)

In [61]:
np.where(mels1!=0)

(array([   4,   18,   24, ..., 9970, 9972, 9977]),)

In [64]:
_, mels1 = operators[0].get_conn_flattened(P[-1,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real

In [67]:
P[-1,:,:]

array([[ 1, -1, -1, ..., -1, -1,  1],
       [-1,  1,  1, ..., -1, -1, -1],
       [-1, -1,  1, ..., -1,  1, -1],
       ...,
       [ 1,  1,  1, ..., -1, -1,  1],
       [-1, -1, -1, ...,  1,  1,  1],
       [-1, -1, -1, ..., -1,  1, -1]], dtype=int8)

In [69]:
_, mels2_ = operators[3].get_conn_flattened(P_list_, sections2)
mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real

In [76]:
np.where(mels2[-10])

(array([], dtype=int64),)

In [77]:
length = [12, 12]
g = nk.graph.Graph(nodes = [i for i in range(length[0] * length[1] * 2)])
hi = nk.hilbert.Spin(s=0.5, graph=g)
hex_ = nk.machine.new_hex(np.array(length))


l = []
for i in range(length[0]):
    for j in range(length[1]):
        l.append([i, j])
l = np.array(l)
a = 0
a_ = [a for _ in range(np.prod(length))]
a = np.array(a_)



edges, colors = hex_.dimer_corr(l,a)
operators = f.return_dimer_operator(hi, edges, colors)



P_list= cal_mels(P, T, t_list)
P_list_ = P_list.reshape(-1,P_list.shape[-1])
num_samples = (P_list[:,:,0]!=0).sum(axis=1)

sections1 = np.arange(P.shape[1])
sections2 = np.zeros(P_list_.shape[0])

_, mels1 = operators[0].get_conn_flattened(P[0,:,:], sections1)
sub1 = mels1.mean().real
mels1 = mels1.real


dimer_corr = np.zeros((length[0],length[1],t_list.shape[0]))
dimer_std = np.zeros((length[0],length[1],t_list.shape[0]))

for l1 in range(length[0]):
    for l2 in range(length[1]):
        
        print(l1,l2)
        
        sub2 = operators[l1 * length[1] + l2].get_conn_flattened(P[0,:,:], sections1)[1].mean().real
        
        _, mels2_ = operators[l1 * length[1] + l2].get_conn_flattened(P_list_, sections2)
        mels2 = mels2_.reshape(P_list.shape[0], P_list.shape[1]).real
        dimer_corr[l1,l2] = (mels2 * mels1).sum(axis=1)/num_samples - sub1*sub2
        dimer_std[l1,l2] = (mels2 * mels1).std(axis=1) 

0 0
0 1
0 2


<ipython-input-77-afad5d2ed222>:47: RuntimeWarning: invalid value encountered in true_divide
  dimer_corr[l1,l2] = (mels2 * mels1).sum(axis=1)/num_samples - sub1*sub2


0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
3 0
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
4 0
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
5 0
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
6 0
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
7 0
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
11 11


In [27]:
mels2 = operators[1].get_conn_flattened(P_list_, sections2)[1].real.reshape(P_list.shape[0], P_list.shape[1])

In [35]:
dimer_corr[0,1]

array([-0.02744887, -0.02283734, -0.0176729 , -0.01520049, -0.01145983,
       -0.0099783 , -0.00740561, -0.00804266, -0.00675654, -0.00827882,
       -0.00532031, -0.00679535, -0.0071737 , -0.00997976, -0.00825695,
       -0.00826282, -0.00646038, -0.00377089, -0.00449202, -0.00880852,
       -0.00693605, -0.00658448, -0.00347626, -0.00387323,  0.00380113,
        0.00380113,  0.00519208,  0.01202482,  0.006259  ,  0.00779342,
        0.00703389,  0.00128677,  0.00523087,  0.00218076, -0.00180784,
       -0.00910024, -0.00764689, -0.00472159, -0.00113308, -0.01157585,
       -0.01020749, -0.00744887, -0.00570974, -0.00180784,  0.0392178 ,
        0.05255113,  0.06346022,  0.07255113,  0.09019819,  0.0392178 ,
        0.04947421,  0.06346022,  0.06346022,  0.06346022,  0.06346022,
        0.07255113,  0.09755113,  0.09755113,  0.09755113,  0.11540828,
        0.1392178 ,  0.1392178 ,  0.17255113, -0.02744887, -0.02744887,
       -0.02744887, -0.02744887, -0.02744887, -0.02744887, -0.02

In [38]:
y = np.log(dimer_corr[0,0])
x = np.arange(y.shape[0])

In [39]:
from plotly import graph_objects as go

In [19]:
corr = np.load(parentdir+'/save/corr/h=1.0V=0.9l=[12, 12]/n=1.0e+05_a=0.npy', allow_pickle=True)

In [37]:
corr[0][0, 0]

array([0.13948817, 0.13780791, 0.13618765, 0.13448737, 0.13283711,
       0.13115684, 0.12976662, 0.12804634, 0.12658611, 0.12494585,
       0.12330861, 0.12142977, 0.12003395, 0.11834529, 0.11676785,
       0.11507865, 0.11332611, 0.11176884, 0.11016684, 0.10886611,
       0.10723404, 0.10593506, 0.10439379, 0.10298183, 0.10180217,
       0.10036872, 0.09879864, 0.09744832, 0.09627484, 0.09503878,
       0.09369677, 0.09246983, 0.09124847, 0.08978891, 0.08863475,
       0.08733005, 0.08614269, 0.08501287, 0.08416451, 0.08311135,
       0.08215694, 0.08106294, 0.07970519, 0.0786521 , 0.07766927,
       0.07659217, 0.07565623, 0.0744346 , 0.07368558, 0.07299079,
       0.07221516, 0.07155534, 0.07058786, 0.0698754 , 0.06906118,
       0.06821316, 0.06743046, 0.0667573 , 0.06601627, 0.06551748,
       0.06468428, 0.06392034, 0.06311808, 0.0626462 , 0.06230031,
       0.06180779, 0.06140617, 0.06106246, 0.0603505 , 0.05980264,
       0.05915265, 0.05852972, 0.05813189, 0.0581158 , 0.05719

In [33]:
corr[3]

array([99984, 99984, 99984, 99984, 99984, 99984, 99984, 99984, 99984,
       99984, 99982, 99981, 99978, 99970, 99961, 99953, 99933, 99916,
       99902, 99887, 99865, 99848, 99817, 99786, 99753, 99714, 99679,
       99628, 99571, 99531, 99470, 99404, 99332, 99247, 99166, 99048,
       98957, 98858, 98741, 98615, 98480, 98331, 98178, 98013, 97827,
       97649, 97460, 97285, 97097, 96874, 96666, 96443, 96213, 95994,
       95728, 95491, 95237, 94964, 94681, 94427, 94154, 93838, 93556,
       93249, 92942, 92628, 92295, 91989, 91644, 91305, 90988, 90661,
       90337, 89971, 89621, 89249, 88872, 88498, 88140, 87767, 87402,
       87017, 86650, 86214, 85816, 85386, 84968, 84559, 84121, 83726,
       83276, 82874, 82461, 82040, 81607, 81150, 80762, 80310, 79837,
       79413, 78949, 78487, 78010, 77578, 77130, 76662, 76211, 75751,
       75296, 74826, 74363, 73875, 73368, 72864, 72364, 71864, 71374,
       70910, 70465, 69971, 69463, 69013, 68537, 68058, 67591, 67093,
       66646, 66205,

In [16]:
parentdir

'/home/keisuke/Documents/Research/sshfs/DimerMaster'

In [16]:
fig = go.Figure()
y = np.log(np.abs(dimer_corr[0,0]))

fig.add_trace(
    go.Scatter(x = t_list, y = y)
)
fig.show()

NameError: name 'go' is not defined

In [47]:
import scipy.fft as fft
fft.fft2(dimer_corr).shape

(12, 12, 100)

In [52]:
a = np.arange(16).reshape(4,4)

In [54]:
fft.fft2(a)

array([[120. -0.j,  -8. +8.j,  -8. -0.j,  -8. -8.j],
       [-32.+32.j,   0. +0.j,   0. +0.j,   0. -0.j],
       [-32. -0.j,   0. +0.j,   0. -0.j,   0. -0.j],
       [-32.-32.j,   0. +0.j,   0. -0.j,   0. -0.j]])

In [65]:
(a * np.array([1,1j,-1,-1j])).sum()

(-8-8j)

In [73]:
n = 10
L = 16
state = np.zeros(L, dtype=np.int8)
for i in range(L):
    state[i] = n // 2**(L-i)
    n %= 2**(L-i)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1], dtype=int8)

In [75]:
dft_matrix = np.fft.fft(np.eye(10))

In [77]:
dft_matrix[0]

array([1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
       1.+0.j, 1.+0.j])